In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# ==============================================================================
# EJERCICIO 1: APRENDIZAJE DE REGLAS MATEMÁTICAS (REF: FIGURA 6.1)
# ==============================================================================
# Objetivo: Demostrar cómo ML aprende "Reglas" a partir de "Datos" y "Respuestas".
# Problema: Aprender a multiplicar matrices 2x2.
# Entrada: 8 números (dos matrices 2x2 aplanadas).
# Salida: 4 números (matriz resultado 2x2 aplanada).

print("--- GENERANDO DATASET ---")
# 1. Generación de Datos
# ------------------------------------------------------------------------------
NUM_SAMPLES = 50000
RANGO = 20 # Enteros entre -20 y 20

# Generamos matrices A y B aleatorias
# Forma: (50000, 2, 2)
A = np.random.randint(-RANGO, RANGO + 1, size=(NUM_SAMPLES, 2, 2))
B = np.random.randint(-RANGO, RANGO + 1, size=(NUM_SAMPLES, 2, 2))

# Calculamos el resultado real (Ground Truth) usando la regla analítica (np.matmul)
C = np.matmul(A, B)

# 2. Preprocesamiento
# ------------------------------------------------------------------------------
# Las redes neuronales funcionan mejor con datos aplanados y normalizados.
# Aplanamos: Cada entrada será un vector de 8 elementos (4 de A + 4 de B)
# Salida: Vector de 4 elementos (elementos de C)

X = np.concatenate([A.reshape(NUM_SAMPLES, -1), B.reshape(NUM_SAMPLES, -1)], axis=1)
y = C.reshape(NUM_SAMPLES, -1)

print(f"Dimensiones de X (Entrada): {X.shape}") # Debería ser (50000, 8)
print(f"Dimensiones de y (Salida):  {y.shape}") # Debería ser (50000, 4)

# Normalización: Es crucial para que la red converja, especialmente con números grandes.
# Como el rango de entrada es aprox [-20, 20], dividimos por 20.
# El rango de salida puede ser mayor (ej: 20*20 + 20*20 = 800), normalizamos aprox.
scale_in = 20.0
scale_out = 800.0 # Estimación burda del máximo valor posible (20*20 + 20*20)

X_norm = X / scale_in
y_norm = y / scale_out

# Split entrenamiento/test
X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, test_size=0.2, random_state=42)

# 3. Diseño del Modelo
# ------------------------------------------------------------------------------
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(8,)), # 8 Entradas
    tf.keras.layers.Dense(64, activation='relu'), # Capa oculta con capacidad suficiente
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4, activation='linear')  # 4 Salidas (Regresión lineal, sin activación acotada)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 4. Entrenamiento
# ------------------------------------------------------------------------------
print("\n--- ENTRENANDO MODELO (Buscando la 'Regla') ---")
history = model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0, validation_split=0.1)
print("Entrenamiento completado.")

# 5. Evaluación y Comparación
# ------------------------------------------------------------------------------
print("\n--- PRUEBA CON 10 EJEMPLOS NUEVOS ---")

# Generamos 10 ejemplos nuevos aleatorios para probar
A_new = np.random.randint(-RANGO, RANGO + 1, size=(10, 2, 2))
B_new = np.random.randint(-RANGO, RANGO + 1, size=(10, 2, 2))
C_real = np.matmul(A_new, B_new)

# Preparamos para la red
X_new = np.concatenate([A_new.reshape(10, -1), B_new.reshape(10, -1)], axis=1)
X_new_norm = X_new / scale_in

# Predicción
pred_norm = model.predict(X_new_norm)
# Des-normalizamos la predicción
pred_real = pred_norm * scale_out

# Mostrar resultados
print(f"{'Real (Analítico)':<35} | {'Predicción (ML)':<35} | {'Dif Abs (Error)'}")
print("-" * 90)

errores = []
for i in range(10):
    real_flat = C_real[i].flatten()
    pred_flat = pred_real[i]

    # Redondeamos la predicción a enteros para comparar visualmente mejor
    pred_rounded = np.round(pred_flat).astype(int)

    diff = np.abs(real_flat - pred_flat)
    errores.append(np.mean(diff))

    print(f"{str(real_flat):<35} | {str(pred_rounded):<35} | {np.mean(diff):.2f}")

print("-" * 90)
print(f"Error medio absoluto promedio: {np.mean(errores):.4f}")

# ==============================================================================
# CONCLUSIONES
# ==============================================================================
"""
CONCLUSIONES DEL EXPERIMENTO:

1. Aproximación vs Exactitud:
   A diferencia de la programación tradicional (método analítico) que da resultados
   exactos siempre, el modelo de ML realiza una 'aproximación de la función'.
   Aunque los resultados son muy cercanos (el error suele ser bajo), rara vez
   son exactamente iguales en punto flotante, aunque al redondear coinciden.

2. Generalización (Figura 6.1):
   El modelo ha logrado inferir la relación compleja (filas por columnas)
   sin que nosotros programáramos explícitamente los bucles de multiplicación.
   Ha extraído las 'reglas' a partir de los 50,000 ejemplos.

3. Dependencia de los Datos:
   Si probáramos con números fuera del rango [-20, 20] (ej: 1000), el modelo
   probablemente fallaría porque la red neuronal no extrapola bien fuera del
   rango de normalización aprendido. La fórmula analítica, en cambio, es universal.
"""